# Homework #3

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = "https://raw.githubusercontent.com/alexeygrigorev/datasets/master/AB_NYC_2019.csv"

In [3]:
!wget $data -O data.csv

--2021-09-26 18:12:40--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/AB_NYC_2019.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7077973 (6.8M) [text/plain]
Saving to: ‘data.csv’

data.csv            100%[===================>]   6.75M  6.57MB/s    in 1.0s    

2021-09-26 18:12:42 (6.57 MB/s) - ‘data.csv’ saved [7077973/7077973]



In [4]:
df = pd.read_csv('data.csv')
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [5]:
features = [
  'neighbourhood_group',
  'room_type',
  'latitude',
  'longitude',
  'price',
  'minimum_nights',
  'number_of_reviews',
  'reviews_per_month',
  'calculated_host_listings_count',
  'availability_365'
]

In [6]:
df = df[features].copy()
df.isnull().sum()

neighbourhood_group                   0
room_type                             0
latitude                              0
longitude                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64

In [7]:
df.reviews_per_month = df.reviews_per_month.fillna(0)

average = 152
df['above_average'] = df.price >= average
df['above_average'] = df['above_average'].astype(int)

df.isnull().sum()

neighbourhood_group               0
room_type                         0
latitude                          0
longitude                         0
price                             0
minimum_nights                    0
number_of_reviews                 0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
above_average                     0
dtype: int64

# Question 1
What is the most frequent observation (mode) for the column 'neighbourhood_group'?

In [8]:
df.neighbourhood_group.value_counts()

Manhattan        21661
Brooklyn         20104
Queens            5666
Bronx             1091
Staten Island      373
Name: neighbourhood_group, dtype: int64

# Answer 1
Manhattan

# Split the data
* Split your data in train/val/test sets, with 60%/20%/20% distribution.
* Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.
* Make sure that the target value ('price') is not in your dataframe.

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [11]:
len(df_train), len(df_val), len(df_test)

(29337, 9779, 9779)

In [12]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_full_train = df_full_train.reset_index(drop=True)

y_train = df_train.above_average.values
y_val = df_val.above_average.values
y_test = df_test.above_average.values

y_train_linear = np.log1p(df_train.price.values)
y_val_linear = np.log1p(df_val.price.values)
y_test_linear = np.log1p(df_test.price.values)

del df_train['price']
del df_val['price']
del df_test['price']

y_val

array([0, 0, 1, ..., 0, 0, 0])

In [13]:
df_full_train.dtypes

neighbourhood_group                object
room_type                          object
latitude                          float64
longitude                         float64
price                               int64
minimum_nights                      int64
number_of_reviews                   int64
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
above_average                       int64
dtype: object

In [14]:
numerical = [
  'latitude',
  'longitude',
  'minimum_nights',
  'number_of_reviews',
  'reviews_per_month',
  'calculated_host_listings_count',
  'availability_365'
]
categorical = [
  'neighbourhood_group',
  'room_type'
]

# Question 2
* Create the correlation matrix for the numerical features of your train dataset.
  * In a correlation matrix, you compute the correlation coefficient between every pair of features in the dataset.
* What are the two features that have the biggest correlation in this dataset?

In [15]:
df_train[numerical].corr()

,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
latitude,1.000000,0.080301,0.027441,-0.006246,-0.007159,0.019375,-0.005891
longitude,0.080301,1.000000,-0.060660,0.055084,0.134642,-0.117041,0.083666
minimum_nights,0.027441,-0.060660,1.000000,-0.076020,-0.120703,0.118647,0.138901
number_of_reviews,-0.006246,0.055084,-0.076020,1.000000,0.590374,-0.073167,0.174477
reviews_per_month,-0.007159,0.134642,-0.120703,0.590374,1.000000,-0.048767,0.165376
calculated_host_listings_count,0.019375,-0.117041,0.118647,-0.073167,-0.048767,1.000000,0.225913
availability_365,-0.005891,0.083666,0.138901,0.174477,0.165376,0.225913,1.000000


# Answer 2
reviews_per_month and number_of_reviews: 0.584935

# Make price binary
* We need to turn the price variable from numeric into binary.
* Let's create a variable above_average which is 1 if the price is above (or equal to) 152.

In [16]:
df.head()

,neighbourhood_group,room_type,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,above_average
0,Brooklyn,Private room,40.64749,-73.97237,149,1,9,0.21,6,365,0
1,Manhattan,Entire home/apt,40.75362,-73.98377,225,1,45,0.38,2,355,1
2,Manhattan,Private room,40.80902,-73.94190,150,3,0,0.00,1,365,0
3,Brooklyn,Entire home/apt,40.68514,-73.95976,89,1,270,4.64,1,194,0
4,Manhattan,Entire home/apt,40.79851,-73.94399,80,10,9,0.10,1,0,0


# Question 3
* Calculate the mutual information score with the (binarized) price for the two categorical variables that we have. Use the training set only.
* Which of these two variables has bigger score?
* Round it to 2 decimal digits using round(score, 2)

In [17]:
from sklearn.metrics import mutual_info_score
df_train

,neighbourhood_group,room_type,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,above_average
0,Brooklyn,Entire home/apt,40.72760,-73.94495,3,29,0.70,13,50,0
1,Manhattan,Private room,40.70847,-74.00498,1,0,0.00,1,7,0
2,Bronx,Entire home/apt,40.83149,-73.92766,40,0,0.00,1,0,0
3,Brooklyn,Entire home/apt,40.66448,-73.99407,2,3,0.08,1,0,0
4,Manhattan,Private room,40.74118,-74.00012,1,48,1.80,2,67,0
...,...,...,...,...,...,...,...,...,...,...
29332,Brooklyn,Private room,40.71748,-73.95685,6,5,0.13,1,0,0
29333,Brooklyn,Private room,40.66397,-73.98538,1,7,0.17,2,0,0
29334,Manhattan,Private room,40.79994,-73.97001,1,1,0.64,1,88,1
29335,Brooklyn,Private room,40.69585,-73.96344,60,0,0.00,1,0,0


In [18]:
for c in categorical:
  print(c, pd.unique(df_train[c]))

neighbourhood_group ['Brooklyn' 'Manhattan' 'Bronx' 'Queens' 'Staten Island']
room_type ['Entire home/apt' 'Private room' 'Shared room']


In [19]:
def mutual_info_price_score(series):
  return round(mutual_info_score(series, df_full_train.above_average),2)

In [20]:
mi = df_full_train[categorical].apply(mutual_info_price_score)
mi.sort_values(ascending=False)

room_type              0.14
neighbourhood_group    0.05
dtype: float64

# Answer 3
room_type

# Question 4
* Now let's train a logistic regression
* Remember that we have two categorical variables in the data. Include them using one-hot encoding.
* Fit the model on the training dataset.
  * To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
  * model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)
* Calculate the accuracy on the validation dataset and rount it to 2 decimal digits.

### One Hot Encoding

In [21]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

In [22]:
dv = DictVectorizer(sparse=False)

In [23]:
train_dicts = df_train[categorical+numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)
X_train.shape


(29337, 15)

In [24]:
val_dicts = df_val[categorical+numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)
X_val.shape

(9779, 15)

In [25]:
model = LogisticRegression(solver='lbfgs', max_iter=10000, C=1.0, random_state=42)
model.fit(X_train, y_train)

LogisticRegression(max_iter=10000, random_state=42)

In [26]:
model.intercept_[0]

-0.05920336776130328

In [27]:
model.coef_[0].round(3)

array([ 3.000e-03,  4.000e-03, -5.620e+00, -3.057e+00, -1.100e-02,
       -1.830e-01,  1.570e-01,  1.596e+00, -1.200e-02, -1.614e+00,
       -3.000e-03, -4.200e-02,  1.985e+00, -7.930e-01, -1.248e+00])

In [28]:
y_pred = model.predict_proba(X_val)[:, 1]
y_pred

array([0.02892384, 0.59603304, 0.42585817, ..., 0.11352213, 0.03456324,
       0.5324363 ])

In [29]:
price_decision = (y_pred >= 0.5)

In [30]:
initial_accuracy = (y_val == price_decision).mean().round(2)
initial_accuracy

0.79

# Answer 4
0.79

# Question 5
* We have 9 features: 7 numerical features and 2 categorical.
* Let's find the least useful one using the feature elimination technique.
* Train a model with all these features (using the same parameters as in Q4).
* Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
* For each feature, calculate the difference between the original accuracy and the accuracy without the feature.
* Which of following feature has the smallest difference?
  * neighbourhood_group
  * room_type
  * number_of_reviews
  * reviews_per_month
> note: the difference doesn't have to be positive

In [31]:
dict(zip(dv.get_feature_names(), model.coef_[0].round(3)))

{'availability_365': 0.003,
 'calculated_host_listings_count': 0.004,
 'latitude': -5.62,
 'longitude': -3.057,
 'minimum_nights': -0.011,
 'neighbourhood_group=Bronx': -0.183,
 'neighbourhood_group=Brooklyn': 0.157,
 'neighbourhood_group=Manhattan': 1.596,
 'neighbourhood_group=Queens': -0.012,
 'neighbourhood_group=Staten Island': -1.614,
 'number_of_reviews': -0.003,
 'reviews_per_month': -0.042,
 'room_type=Entire home/apt': 1.985,
 'room_type=Private room': -0.793,
 'room_type=Shared room': -1.248}

In [32]:
to_remove = [
  'neighbourhood_group',
  'room_type',
  'number_of_reviews',
  'reviews_per_month'
]

all = numerical + categorical

In [33]:
def feature_elimination(set):
  train_dicts = df_train[set].to_dict(orient='records')
  X_train = dv.fit_transform(train_dicts)

  val_dicts = df_val[set].to_dict(orient='records')
  X_val = dv.transform(val_dicts)
  X_val.shape

  model = LogisticRegression(solver='lbfgs', max_iter=10000, C=1.0, random_state=42)
  model.fit(X_train, y_train)

  y_pred = model.predict_proba(X_val)[:, 1]
  price_decision = (y_pred >= 0.5)
  accuracy = (y_val == price_decision).mean()
  
  return accuracy
  

In [34]:
for removed_feature in to_remove:
  cur_accuracy = feature_elimination(list(set(all) ^ set([removed_feature])))
  print(removed_feature, initial_accuracy - cur_accuracy)

neighbourhood_group 0.03900296553839866
room_type 0.06129563350035794
number_of_reviews -0.0017987524286736667
reviews_per_month -0.0005716330913181356


# Answer 5
reviews_per_month

# Question 6
* For this question, we'll see how to use a linear regression model from Scikit-Learn
* We'll need to use the original column 'price'. Apply the logarithmic transformation to this column.
* Fit the Ridge regression model on the training data.
* This model has a parameter alpha. Let's try the following values: [0, 0.01, 0.1, 1, 10]
* Which of these alphas leads to the best RMSE on the validation set? Round your RMSE scores to 3 decimal digits.

If there are multiple options, select the smallest alpha.

In [35]:
from sklearn.linear_model import Ridge

In [36]:
alpha_values = [0, 0.01, 0.1, 1, 10]

In [37]:
def rmse(y, y_pred):
  se = (y - y_pred) ** 2
  mse = se.mean()
  return np.sqrt(mse)

In [38]:
for a in alpha_values:
  model = Ridge(a)
  model.fit(X_train, y_train_linear)
  y_pred = model.predict(X_val)
  print(a,',', rmse(y_val_linear, y_pred).round(3))

0 , 0.497
0.01 , 0.497
0.1 , 0.497
1 , 0.497
10 , 0.498


# Answer 6
0